## 주식 데이터 파싱 (한국)

- (1) 한국거래소에서 운영하는 기업공시채널 : https://kind.krx.co.kr/main.do?method=loadInitPage&scrnmode=1

- (2) 마켓 데이터 취급하는 한국 거래소 사이트(본체) : https://km.krx.co.kr/main/main.jsp

- 여기 2개의 사이트에서 현재 주식시장에 상장되어 있는 상장법인목록을 크롤링해보자

    - 위 사이트에 방문에서 조건을 설정하고 csv나 excel로 다운받을 수도 있지만, 지난 주차에 배운 크롤링으로 한번 해본다

    - (2)이 본체 ((2)이 (1)에 데이터를 제공)이다. (1) 사이트 크롤링과 (2) 사이트 크롤링을 모두 해본다. 

    - (1)이 훨씬 쉽지만, (2)에 주식 데이터 뿐만 아니라 활용할 수 있는 데이터가 많기 때문에 (2)도 다루게 되었다.

In [4]:
import pandas as pd

In [5]:
# (1) 사이트 크롤링
# 코스피, 코스닥으로 나눠서 상장법인의 종목코드를 구해본다. 
# 여기서 csv, excel을 다운받을 수 있다. http://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage

# 마켓 타입(market type)에 따라 download url이 다름. 딕셔너리 형태로 지정
stock_type = {
    'kospi': 'stockMkt',
    'kosdaq': 'kosdaqMkt'
}

# 코스피와 코스닥 download url 조합
def get_download_stock(market_type=None):

    # 딕셔너리를 통해 어떤 마켓타입인지 확인하고 market type 파라미터를 지정해준다. 
    market_type = stock_type[market_type]

    # 다운로드 할 때 개발자도구에서 다음 사이트를 요청하는 것을 확인한다.
    download_link = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download'

    # 마켓 타입에 따라 파라미터를 다르게 넘겨줘서 원하는 마켓 목록만 다운받을 수 있다.
    download_link = download_link + '&marketType=' + market_type

    # 이렇게 저장하면 파일을 다운받아서 따로 pandas를 통해 열지 않고 바로 데이터프레임 형태로 저장할 수 있다.
    # 사이트가 다운로드할 내용을 html 구조로 제공해야 다음과 같이 read_html으로 받을 수 있다.
    df = pd.read_html(download_link, header=0)[0]
    return df


# kospi 종목코드 목록 다운로드
def get_download_kospi():

    # download url 함수 호출
    df = get_download_stock('kospi')

    # 오른쪽으로 6자리로 정렬하고 나머지는 0으로 채운다. (다운로드 받을 때 종목코드에 앞쪽 0이 사라짐)
    df.종목코드 = df.종목코드.map('{:06d}'.format)
    return df

# kosdaq 종목코드 목록 다운로드
def get_download_kosdaq():

    # download url 함수 호출
    df = get_download_stock('kosdaq')

    # 마찬가지
    df.종목코드 = df.종목코드.map('{:06d}'.format)
    return df

# kospi, kosdaq 종목코드 각각 다운로드
kospi_df = get_download_kospi()
기kosdaq_df = get_download_kosdaq()

In [7]:
# 코스피와 코스닥 5개씩 출력해보기
print(kospi_df.head(5))
print(kosdaq_df.head(5))

# 코스피와 코스닥 목록을 합친다.
code_df = pd.concat([kospi_df, kosdaq_df])

# 필요한 칼럼인 회사명, 종목코드만 가져온다.
list_df = code_df[['회사명', '종목코드']]

# 칼럼을 한글이 아닌 영어로 간단하게 바꾼다. '회사명' = name, 종목코드 = 'code'
list_df = list_df.rename(columns={'회사명': 'name', '종목코드': 'code'})

회사명    종목코드             업종                주요제품         상장일  결산월   대표자명  \
0    JS전선  005560  절연선 및 케이블 제조업  선박선,고무선,전력선,통신선 제조  2007-11-12  12월    이익희   
1   거북선2호  101380            NaN         운송장비(선박) 임대  2008-04-25  12월    신주선   
2   거북선6호  114140            NaN                 NaN  2009-10-01  12월    김연신   
3   교보메리츠  064900            NaN           부동산 투자,운용  2002-01-30  12월  김 상 진   
4  국제관광공사  028780            NaN                 NaN  1966-03-18  12월    NaN   

                       홈페이지       지역  
0  http://www.jscable.co.kr     충청남도  
1                       NaN    부산광역시  
2                       NaN  제주특별자치도  
3                       NaN    서울특별시  
4                       NaN      NaN  
         회사명    종목코드                   업종                         주요제품  \
0         3R  037730                  NaN               DVR(디지털영상저장장치)   
1     3노드디지탈  900010  회사 본부 및 경영 컨설팅 서비스업                         지주회사   
2  IBKS제7호스팩  276920           금융 지원 서비스업                       기업인수목

In [10]:
# 특정 회사의 종목코드를 갖고와보자.

# 회사명으로 주식 종목 코드를 획득할 수 있도록 하는 함수
def get_code(df, name):

    #query로 name(회사명)이 파라미터로 넘겨준 name과 같으면 code열에서 종목코드 찾고 스트링으로 출력하는데 인덱스 번호 빼고 출력(false)
    code = df.query("name=='{}'".format(name))['code'].to_string(index=False)
    # 위와같이 code명을 가져오면 앞에 공백이 붙어있는 상황이 발생하여 앞뒤로 strip() 하여 공백 제거
    code = code.strip()
    return code


# 삼성전자의 종목코드를 갖고와보자.
code = get_code(list_df, '삼성전자')
print(code)

005930


In [11]:
# (2) 사이트 크롤링
# 여기서 csv, excel을 다운받을 수 있다. http://marketdata.krx.co.kr/mdi#document=040601

from io import BytesIO 
from urllib.request import urlopen, Request
import pandas as pd
import pickle
# import sqlite3

# 파일을 다운로드하는 것을 클릭하면 다음 링크를 호출함. 
# 링크를 보면 OTP가 생성됨을 확인할 수 있고, 
# 다운로드할 내용의 여러 조건들이 파라미터로 지정됨을 알 수 있음. 개발자도구로 확인
# 마찬가지로 헤더도 같이 넘겨줘야함
req_url='http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?name=fileDown&filetype=xls&url=MKD/04/0406/04060100/mkd04060100_01&market_gubun=ALL&isu_cdnm=%EC%A0%84%EC%B2%B4&isu_cd=&isu_nm=&isu_srt_cd=&sort_type=A&std_ind_cd=&par_pr=&cpta_scl=&sttl_trm=&lst_stk_vl=1&in_lst_stk_vl=&in_lst_stk_vl2=&cpt=1&in_cpt=&in_cpt2=&isu_cdnm=%EC%A0%84%EC%B2%B4&isu_cd=&mktpartc_no=&isu_srt_cd=&pagePath=%2Fcontents%2FMKD%2F04%2F0406%2F04060100%2FMKD04060100.jsp'

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

# requests 라이브러리를 사용하면, requests.get(url, headers=headers) 하면 되지만, 표준라이브러리 urllib 사용해봄
req1 = Request(req_url,headers=headers)
result = urlopen(req1)

# 최종적으로 암호화된 코드가 리턴되는데 바이트형태라 디코딩해줌
code = result.read().decode()

# 파일을 다운로드하는 링크에 otp로 암호화된 코드가 포함이 됨. 링크 url에 ?code= 에 코드 파라미터 값을 넘겨줌
req_url2 = 'http://file.krx.co.kr/download.jspx?code=' + code 


# 마찬가지로 다운로드 링크를 요청할 때 헤더를 넘겨줘야하는데 referer도 같이 넘겨줘야 한다.
headers = {'Referer': 'http://marketdata.krx.co.kr/mdi', 'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'} 

req2 = Request(req_url2, headers=headers) # 헤더(리퍼러) 추가 

# 바이트 스트림 형태의 데이터
byte_data = urlopen(req2).read()

# 여기는 html구조로 제공되는 것이 아니라 byte 형태로 excel 구조로 내용을 만들어줘서, read_excel로 읽어온다.
# byte_data는 BytesIO 객체로 바꿔서 인 메모리 바이트 스트림으로 바꾸고 read_excel로 읽어온다.
df = pd.read_excel(BytesIO(byte_data))

# 갯수만큼 숫자로 붙여진 인덱스는 삭제
del df['번호'] 

# 칼럼별로 타입지정과 전처리를 수행한다. 숫자를 나타내는 문자열에 , 가 있으면 지우고 수치형자료로 변환한다.
df['종목코드'] = df['종목코드'].map(lambda x: f'{x:0>6}') # 길이 6 오른쪽으로 정렬하고 남는 공간 0으로 채움.
df['업종코드'] = df['업종코드'].map(lambda x: f'{x:0>6}') 
df['상장주식수(주)'] = df['상장주식수(주)'].str.replace(",", "") 
df['상장주식수(주)'] = pd.to_numeric(df['상장주식수(주)'])  
df['자본금(원)'] = df['자본금(원)'].str.replace(",", "") 
df['자본금(원)'] = pd.to_numeric(df['자본금(원)']) 

In [12]:
print(df.head(5))

종목코드     기업명    업종코드               업종  상장주식수(주)       자본금(원) 액면가(원)  \
0  060310      3S  032902    특수 목적용 기계 제조업  44802511  22401255500    500   
1  095570  AJ네트웍스  147603  산업용 기계 및 장비 임대업  46822295  46822295000  1,000   
2  006840   AK홀딩스  116409           기타 금융업  13247561  66237805000  5,000   
3  054620  APS홀딩스  116409           기타 금융업  20394221  10197110500    500   
4  265520   AP시스템  032902    특수 목적용 기계 제조업  14480227   7240113500    500   

     통화구분          대표전화                              주소  
0  원(KRW)   02-896-9474        서울특별시 금천구  시흥대로71길 30-1   
1  원(KRW)  02-6363-9999    서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)  
2  원(KRW)   02-768-2923            서울특별시 마포구 양화로 188 -   
3  원(KRW)  031-776-1800  경기도 화성시 동탄면 동탄산단9길 23-12 (동탄면)  
4  원(KRW)  031-379-2700  경기도 화성시  동탄면 동탄산단8길 15-5 &nbsp  


In [13]:
# pickle 로 저장 (추천)
df.to_pickle('상장법인목록.pkl')

# pickle 읽어오기
df = pd.read_pickle('상장법인목록.pkl')

print(df.head(5))

# input_output.ipynb에서 다뤘던 것처럼, excel, csv, database등 다양한 형태로 자료를 관리할 수 있다.

종목코드     기업명    업종코드               업종  상장주식수(주)       자본금(원) 액면가(원)  \
0  060310      3S  032902    특수 목적용 기계 제조업  44802511  22401255500    500   
1  095570  AJ네트웍스  147603  산업용 기계 및 장비 임대업  46822295  46822295000  1,000   
2  006840   AK홀딩스  116409           기타 금융업  13247561  66237805000  5,000   
3  054620  APS홀딩스  116409           기타 금융업  20394221  10197110500    500   
4  265520   AP시스템  032902    특수 목적용 기계 제조업  14480227   7240113500    500   

     통화구분          대표전화                              주소  
0  원(KRW)   02-896-9474        서울특별시 금천구  시흥대로71길 30-1   
1  원(KRW)  02-6363-9999    서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)  
2  원(KRW)   02-768-2923            서울특별시 마포구 양화로 188 -   
3  원(KRW)  031-776-1800  경기도 화성시 동탄면 동탄산단9길 23-12 (동탄면)  
4  원(KRW)  031-379-2700  경기도 화성시  동탄면 동탄산단8길 15-5 &nbsp  


## 상장법인목록을 이용해서 종목의 주가 데이터 가져오기 (네이버)

- 네이버 과거 주가는 수정주가로 반영되어 있지 않고, 페이지를 돌면서 크롤링 해야 하므로 비효율적

- 네이버 금융에서 제공하는 차트는 수정주가로 반영되어 있는데, 이것을 가져올 수 있지 않을까?

    - 다만 확인해본 결과 정확한 데이터는 아니므로 투자에 참고하기에는 꺼림칙하다.

    - xml 데이터 구조이다. 파이썬에서는 서드파티 라이브러리로 lxml을 제공하는데 xml는 beautifulsoup과 lxml 이용해서 처리할 수도 있지만 여기서는 표준 xml 패키지를 사용해보겠다.

    - xml 이란 ? : https://ko.wikipedia.org/wiki/XML



In [14]:
import pandas
from urllib.request import urlopen 
# XML을 객체화 하기 위해서는 xml.etree.ElementTree이 필요하므로 import 합니다.
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import datetime


# 네이버 차트 수정주가 가져오기. 앞에서 구한 특정 종목의 종목코드를 가져온다.
code = get_code(list_df, '삼성전자')
count = '6000' 
day = 'day'

# 이건 차트 띄울때 개발자 도구 확인하면 사이트 확인할 수 있다.
url = f'https://fchart.stock.naver.com/sise.nhn?symbol={code}&timeframe={day}&count={count}&requestType=0' 

# 요청
r = urlopen(url) 

# 인코딩 해야지 정확한 데이터가 안깨지고 가져와진다. 해당 사이트 인코딩은 EUC-KR
xml_data = r.read().decode('EUC-KR') 

# beautiful soup 이용해서 xml 데이터 정적 크롤링을 수행할 수도 있다. 
# # beautifulsoup으로 가져오기
# xml_data = requests.get(url)
# soup_xml = BeautifulSoup(xml_data.text, 'lxml')
# for i, row in enumerate(soup_xml.find_all('item')):
#   price = row['data'].split('|')
# 
# 하지만 xml데이터를 다루는 다른 방법도 하나 알아보자.

# XML객체로 파싱(Parse)하기
# XML로 파싱하는 방법은 두 가지가 있습니다.
    # 1. xml 파일형태로 존재하는 경우
    # 2. xml 문자열 형태로 존재하는 경우

# 우리는 2 문자열 형태로 존재하는 경우이다. 
# 문자열 값으로 존재하는 경우 fromstring() 메서드에 xml 데이터 문자열을 넘깁니다.

root = ET.fromstring(xml_data)

# xml 데이터를 객체로 파싱한 이후 트리 안에서 각각의 태그를 검색할 수 있다. 
# 태그 검색 : find, findall
print(root.findall('.//item'))

[<Element 'item' at 0x000002238A939E58>, <Element 'item' at 0x000002238A939EF8>, <Element 'item' at 0x000002238A939D18>, <Element 'item' at 0x000002238A9394F8>, <Element 'item' at 0x000002238A939B38>, <Element 'item' at 0x000002238A939638>, <Element 'item' at 0x000002238A939EA8>, <Element 'item' at 0x000002238A939BD8>, <Element 'item' at 0x000002238A939CC8>, <Element 'item' at 0x000002238A939D68>, <Element 'item' at 0x000002238A939318>, <Element 'item' at 0x000002238A939E08>, <Element 'item' at 0x000002238A939C78>, <Element 'item' at 0x000002238A939DB8>, <Element 'item' at 0x000002238A9399A8>, <Element 'item' at 0x000002238A939908>, <Element 'item' at 0x000002238A939138>, <Element 'item' at 0x000002238A939A98>, <Element 'item' at 0x000002238A9390E8>, <Element 'item' at 0x000002238A939278>, <Element 'item' at 0x000002238A939868>, <Element 'item' at 0x000002238A939F98>, <Element 'item' at 0x000002238A939818>, <Element 'item' at 0x00000223880BAEF8>, <Element 'item' at 0x00000223880BA548>,

In [15]:
price_table=[]

# 태그별로 반복문
for i, row in enumerate(root.findall('.//item')):
    
    # 태그의 속성을 이용해서 안에 있는 데이터를 가져온다. 구분자는 | 로 되어있음을 확인한다.
    price = row.attrib['data'].split('|')
    # price = [19900113, 44000, 45800, ...]

    # 2000 이후 데이터만 가져오자
    # 문자열 데이터는 적절하게 변환시켜서 저장한다.
    if price[0] >= "20000101":  
        price[0] = datetime.datetime.strptime(price[0],'%Y%m%d')
        price[1] = int(price[1]) 
        price[2] = int(price[2]) 
        price[3] = int(price[3]) 
        price[4] = int(price[4]) 
        price[5] = int(price[5]) 
        price_table.append(price)

# 데이터프레임으로 저장
price_df=pd.DataFrame(price_table)
price_headers=['Date', 'Open','High','Low','Close', 'Volume']
price_df.columns=price_headers
price_df.set_index('Date',inplace=True)

print(price_df.head(10))

Open  High   Low  Close   Volume
Date                                        
2000-01-04  6000  6110  5660   6110  1483967
2000-01-05  5800  6060  5520   5580  1493604
2000-01-06  5750  5780  5580   5620  1087810
2000-01-07  5560  5670  5360   5540   806195
2000-01-10  5600  5770  5580   5770   937615
2000-01-11  5820  6100  5770   5770  1194974
2000-01-12  5610  5740  5600   5720   584492
2000-01-13  5600  5740  5560   5710   823830
2000-01-14  5720  5880  5680   5830   987576
2000-01-17  6000  6180  5920   6100  1270138


## 네이버 금융의 자료들을 이용하는 방법

- 앞서 네이버 일별 주가, 차트 데이터도 파싱이 가능함을 확인했다.

- 이번에는 네이버 금융에서 데이터를 제공하고 관리하는 방식인 api를 이용해서 갖고오는 방법을 알아보자

    - 여기서는 etf목록을 가져올텐데 다른 데이터에도 사용할 수 있는 방법이다. 사이트 : https://finance.naver.com/sise/etf.nhn

    - 위 사이트에서 개발자도구를 열고 ctrl + f를 해서 ajax를 검색한다.

    - 그러면 /*Ajax 호출을 통한 ETF 리스트 테이블 업데이트*/ 부분이 나오고 여기에서 사이트의 api 주소가 무엇인지 유추할 수 있다.

- 네이버 금융 api는 json 형식의 데이터를 사용한다.

    - JSON은 JavaScript Object Notation의 약자로서 JavaScript 문법에 영향을 받아 개발된 Lightweight한 데이타 표현 방식이다. 
    
    - JSON은 데이타를 교환하는 한 포맷으로서 그 단순함과 유연함 때문에 널리 사용되고 있다. 
    
    - 특히 웹 브라우져와 웹서버 사이에 데이타를 교환하는데 많이 사용되고 있다. 가장 많이 사용되는 JSON 포맷은 Key-Value Pair의 컬렉션이다.

    - Python은 기본적으로 JSON 표준 라이브러리(json)를 제공하고 있는데, "import json" 을 사용하여 JSON 라이브러리를 사용할 수 있다

    - JSON 라이브러리를 사용하면, Python 타입의 Object를 JSON 문자열로 변경할 수 있으며 (JSON 인코딩 : json.dump() 메서드 사용)
    
    - 또한 JSON 문자열을 다시 Python 타입으로 변환할 수 있다 (JSON 디코딩 : json.loads() 메서드 사용).



In [16]:
# 유추한 url은 다음과 같다.

url = 'https://finance.naver.com/api/sise/etfItemList.nhn' 

# 확인해보면 네이버 금융 api는 json 형태를 통해 데이터를 관리하고 있음을 알 수 있다.

#{"resultCode":"success","result":{"etfItemList":[{"itemcode":"069500","etfTabCode":1,"itemname":"KODEX 200","nowVal":29465,"risefall":"5","changeVal":-315,"changeRate":-1.06,"nav":29522.0,"threeMonthEarnRate":8.841,"quant":3542631,"amonut":104730,"marketSum":92063}, ...


import json 
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

# 웹 요청을 하고 사이트 인코딩(CP949)에 맞게 디코딩해준다. 
raw_data = urlopen(url).read().decode('CP949') 

# json데이터를 파이썬에서 사용할 수 있게 디코딩 해준다. json.loads()메서드 사용
json_data = json.loads(raw_data)

# json문자열 데이터가 파이썬에서 활용할 수 있는 딕셔너리 형태로 변환이 된다.
print(json_data)

{'resultCode': 'success', 'result': {'etfItemList': [{'itemcode': '069500', 'etfTabCode': 1, 'itemname': 'KODEX 200', 'nowVal': 31325, 'risefall': '5', 'changeVal': -390, 'changeRate': -1.23, 'nav': 31424.0, 'threeMonthEarnRate': 9.2791, 'quant': 5170204, 'amonut': 162520, 'marketSum': 50167}, {'itemcode': '102110', 'etfTabCode': 1, 'itemname': 'TIGER 200', 'nowVal': 31340, 'risefall': '5', 'changeVal': -380, 'changeRate': -1.2, 'nav': 31424.0, 'threeMonthEarnRate': 9.3529, 'quant': 766875, 'amonut': 24093, 'marketSum': 26388}, {'itemcode': '252670', 'etfTabCode': 3, 'itemname': 'KODEX 200선물인버스2X', 'nowVal': 4140, 'risefall': '2', 'changeVal': 90, 'changeRate': 2.22, 'nav': 4173.0, 'threeMonthEarnRate': -20.1971, 'quant': 149233144, 'amonut': 616480, 'marketSum': 23896}, {'itemcode': '122630', 'etfTabCode': 3, 'itemname': 'KODEX 레버리지', 'nowVal': 15020, 'risefall': '5', 'changeVal': -345, 'changeRate': -2.25, 'nav': 15043.0, 'threeMonthEarnRate': 17.0666, 'quant': 40227032, 'amonut': 60

In [17]:
# 딕셔너리 데이터 확인

# etf 코드와 이름만 반복문으로 출력해본다.
for each in json_data['result']['etfItemList']:
    print(each['itemcode'], each['itemname'])

069500 KODEX 200
102110 TIGER 200
252670 KODEX 200선물인버스2X
122630 KODEX 레버리지
153130 KODEX 단기채권
273130 KODEX 종합채권(AA-이상)액티브
102780 KODEX 삼성그룹
157450 TIGER 단기통안채
114800 KODEX 인버스
278530 KODEX 200TR
233740 KODEX 코스닥150 레버리지
148020 KBSTAR 200
214980 KODEX 단기채권PLUS
152100 ARIRANG 200
293180 HANARO 200
105190 KINDEX 200
272580 TIGER 단기채권액티브
278540 KODEX MSCI Korea TR
069660 KOSEF 200
292150 TIGER TOP10
310970 TIGER MSCI Korea TR
261220 KODEX WTI원유선물(H)
251340 KODEX 코스닥150선물인버스
357870 TIGER CD금리투자KIS(합성)
226490 KODEX 코스피
305720 KODEX 2차전지산업
251350 KODEX 선진국MSCI World
133690 TIGER 미국나스닥100
190620 KINDEX 단기통안채
229200 KODEX 코스닥 150
305540 TIGER 2차전지테마
315930 KODEX Top5PlusTR
294400 KOSEF 200TR
306540 HANARO 단기통안채
295040 SMART 200TR
232080 TIGER 코스닥150
192090 TIGER 차이나CSI300
302450 KBSTAR 코스피
139260 TIGER 200 IT
204480 TIGER 차이나CSI300레버리지(합성)
132030 KODEX 골드선물(H)
148070 KOSEF 국고채10년
161510 ARIRANG 고배당주
332500 KINDEX 200TR
332940 HANARO MSCI Korea TR
278240 KBSTAR 코스닥150선물레버리지
314250 KODEX 미국FANG플러

In [20]:
# 앞의 과정을 urllib 표준 라이브러리 말고 requests 라이브러리를 사용하면 훨씬 간단하게 된다.
# 위의 과정을 한줄로 요약할 수 있다.

import requests
json_data = json.loads(requests.get(url).text)

# json_noramlize를 사용하면 json 형태의 딕셔너리 데이터를 바로 데이터프레임 형태로 저장할 수 있다.
df = json_normalize(json_data['result']['etfItemList'])

# 데이터프레임 출력 및 확인
print(df.head(10))


itemcode  etfTabCode              itemname  nowVal risefall  changeVal  \
0   069500           1             KODEX 200   31325        5       -390   
1   102110           1             TIGER 200   31340        5       -380   
2   252670           3      KODEX 200선물인버스2X    4140        2         90   
3   122630           3            KODEX 레버리지   15020        5       -345   
4   153130           6            KODEX 단기채권  102455        5         -5   
5   273130           6  KODEX 종합채권(AA-이상)액티브  110395        2        220   
6   102780           2            KODEX 삼성그룹    7650        5        -85   
7   157450           6           TIGER 단기통안채  100655        3          0   
8   114800           3             KODEX 인버스    5445        2         60   
9   278530           1           KODEX 200TR   10050        5       -110   

   changeRate       nav  threeMonthEarnRate      quant  amonut  marketSum  
0       -1.23   31424.0              9.2791    5170204  162520      50167  
1       -1.20

## 주가데이터 패키지를 이용하여 가져오기

- 위에서 네이버 차트의 주가데이터를 가져올 수 있었다 (네이버는 상당히 많은 양의 과거자료를 포함하고 있음)

- 하지만 한국주식 한정이라는 점도 있고 무엇보다 사용하기 편하지 않다.

- 이번에는 남들이 만들어 놓은 패키지들을 활용해서 주가 데이터를 가져와보자

    - FinanceDataReader 사용

    - pandas_datareadr 사용

In [22]:
import FinanceDataReader as fdr 

# 삼성전자 5년간 가격데이터. 종목코드, 시작일, 종료일 순. 종료일 없으면 최근것 까지
ss_df = fdr.DataReader('005930', '2015-01-01', '2019-12-31')

# 여기서 종가 Close는 수정주가이다. financedataread는 investing에서 자료를 가져오는데 인베스팅이 수정주가를 기준으로 종가를 표시한다.
print(ss_df)

Open   High    Low  Close    Volume    Change
Date                                                      
2015-01-02  26799  26799  26540  26600    175499  0.002261
2015-01-05  26720  26720  26260  26660    202790  0.002256
2015-01-06  26300  26340  25760  25900    304710 -0.028507
2015-01-07  25880  26220  25640  26140    286455  0.009266
2015-01-08  26780  26780  26199  26280    289552  0.005356
...           ...    ...    ...    ...       ...       ...
2019-12-23  56100  56400  55100  55500   9839252 -0.008929
2019-12-24  55600  55700  54800  55000  11868463 -0.009009
2019-12-26  54700  55400  54400  55400   9645034  0.007273
2019-12-27  55700  56900  55500  56500  12313056  0.019856
2019-12-30  56200  56600  55700  55800   8356767 -0.012389

[1227 rows x 6 columns]


In [24]:
# padas 에서 제공하는 data_reader 모듈을 이용해서도 주가 데이터를 가져올 수 있다.
# 구글, 야후, fred 등등이 있는데 야후 파이낸스 데이터를 가져와보다.
import pandas as pd
import pandas_datareader.data as pdr

# 야후 파이낸스를 판다스에서 중단했어서 그 기능을 사용하기 위해 불러올 모듈
import yfinance as yf
yf.pdr_override()

# 종목코드, 시작일, 종료일 순. 종료일 없으면 최근것 까지
ss_df = pdr.get_data_yahoo('005930.KS', '2000-01-04')  

# OHLCV와 수정주가도 따로 표시됨을 확인할 수 있음.
print(ss_df)

[*********************100%***********************]  1 of 1 completed
               Open     High      Low    Close     Adj Close    Volume
Date                                                                  
2000-01-04   6000.0   6110.0   5660.0   6110.0      0.090599  74195000
2000-01-05   5800.0   6060.0   5520.0   5580.0      0.082740  74680000
2000-01-06   5750.0   5780.0   5580.0   5620.0      0.083333  54390000
2000-01-07   5560.0   5670.0   5360.0   5540.0      0.082147  40305000
2000-01-10   5600.0   5770.0   5580.0   5770.0      0.085558  46880000
...             ...      ...      ...      ...           ...       ...
2020-09-03  55600.0  56700.0  55500.0  56400.0  56400.000000  28384920
2020-09-04  55200.0  55800.0  55100.0  55600.0  55600.000000  22139109
2020-09-07  56100.0  57300.0  55800.0  56500.0  56500.000000  18685880
2020-09-08  57400.0  58700.0  57200.0  58700.0  58700.000000  31517520
2020-09-09  58200.0  59300.0  57800.0  58400.0  58400.000000  30031011

[5194 r